# Pacakge

In [1]:
import os
import tarfile
import pathlib
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import html
import numpy as np
import html
import re
from pandarallel import pandarallel
import unicodedata
import warnings
from collections import Counter  
warnings.simplefilter("ignore")

def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")"))
    return TEXT

def get_csv_size(csv_name):
    get_size = os.path.getsize (os.getcwd() + '/'+csv_name)
    mb_size = get_size/(1024*1024)
    mb_size = round(mb_size,1)
    return mb_size

In [64]:
with open('wiki_relevant.txt', 'r') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

relevant_rules = re.compile(relevant_rules, re.IGNORECASE)

print('relevant_rules:')
print(relevant_rules)

with open('irelevant_keywords.txt', 'r') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

print('\nirelevant_rules:')
print(irelevant_rules)

relevant_rules:
re.compile('\\b\\w+-\\w+ relations|\\babduct|\\babuse|\\bactivist|\\bacts\\b|\\badministration|\\bagainst women|\\bagencies|\\ballegati|\\bambassador|\\banarch|\\barmed|\\barmy|\\barrest|\\bartillery|\\bassassin, re.IGNORECASE)

irelevant_rules:
re.compile('\\b/styles?/|\\b/autos?/|\\b/books?/|\\b/cars?/|\\b/earth/|\\b/food/|\\b/future/|\\b/health/|\\b/life/|\\b/living/|\\b/members?/|\\b/money/|\\b/packages?/|\\b/pictures?/|\\b/popular/|\\b/radio/|\\b/v, re.IGNORECASE)


In [65]:
irelevant_rules

re.compile(r'\b/styles?/|\b/autos?/|\b/books?/|\b/cars?/|\b/earth/|\b/food/|\b/future/|\b/health/|\b/life/|\b/living/|\b/members?/|\b/money/|\b/packages?/|\b/pictures?/|\b/popular/|\b/radio/|\b/videos?/|\baccident|\balbum|\bAMEX|\banimal|\bart\b|\bartist|\bathlet|\baustralian.open|\bAuto.Rac|\bbadminton|\bbaseball|\bbasketball|\bbbcthree|\bbcs.champion|\bbcs.national|\bbeauty|\bbids?\b|\bBillboard|\bBooks? Review|\bbowl\b|\bbowling|\bboxing|\bBraves|\bBrewers|\bBritish.open|\bBroadway|\bbusiness|\bCardinals|\bcareer|\bCBA.Playoffs|\bcelebrit|\bchampinship|\bchampion|\bcinema|\bclimate|\bclinics?\b|\bcomed|\bcommentisfree|\bconcert|\bconsumer|\bCorporat|\bcricket|\bculture|\bcup\b|\bcurrency|\bcyclyng|\bdaytona 500|\bdeals?\b|\bDJIA|\bDollar|\bDow.Jones|\bdragster|\bEconom|\beducation|\bentertainment|\benvironment|\bequestrian|\bexchange|\bExpos|\bfarming|\bfashion|\bfiction|\bfield \w+ goal|\bFIFA\b|\bfilm|\bFINA\b|\bfinalist|\bfinals|\bfinanc|\bfitness|\bFlowery.Gold.Mines|FLOD|\bfoot

In [3]:
domain1 = 'TheNewsInternational|voanew|rfa.org|rferl.org|.wn.com|theepochtimes|irinnews|thenewhumanitarian|'+\
'abcnews|aljazeera|allafrica|apnews|bbc|channelnewsasia|cnbc|cnn|theconversation|dw.com|efe.com|'+\
'theguardian|france24|indiatimes|japantimes|latimes|nbcnews|news24|npr.org|nypost|nytimes|pbs.org|'+\
'politico.com|reuters.com|rfi.fr|scmp|sfgate|tass.com|thenews.com.pk|upi.com|usatoday|washingtonpost|wsj.com|xinhuanet'

domain2 =\
'bbj.hu|menafn.com|www.ft.com|seekingalpha.com|'+\
'www.thefridaytimes.com|www.herald.ie|'+\
'youtube|www.ign2.com|preen|theverge.com|'+\
'phys.org|technology.inquirer|gizmodo.com|engadget.com|arstechnica|techhq|androidpolice|'+\
'macrumors|www.wired.com|www.thurrott.com|seekingalpha|deadspin.com|'+\
'aol.com|eonline.com|www.cnet.com|www.manilachannel.com'

domain3 =\
'http://www.infotag.md|'+\
'http://dailyexplorer.net|'+\
'https://www.notebookcheck.net|'+\
'https://www.androidauthority.com|'+\
'https://www.phonearena.com|'+\
'https://bleacherreport.com|'+\
'https://www.vulture.com|'+\
'http://admin.punchng.com|'+\
'https://www.androidcentral.com|'+\
'https://www.livescience.com|'+\
'www.espn.com|'+\
'seinstrukcii.date'


excude_domain = domain1 +'|'+ domain2 + '|' + domain3
excude_domain = re.compile(excude_domain, re.IGNORECASE)
excude_domain
# keywords_domain = re.compile(keywords_domain, re.IGNORECASE)

re.compile(r'TheNewsInternational|voanew|rfa.org|rferl.org|.wn.com|theepochtimes|irinnews|thenewhumanitarian|abcnews|aljazeera|allafrica|apnews|bbc|channelnewsasia|cnbc|cnn|theconversation|dw.com|efe.com|theguardian|france24|indiatimes|japantimes|latimes|nbcnews|news24|npr.org|nypost|nytimes|pbs.org|politico.com|reuters.com|rfi.fr|scmp|sfgate|tass.com|thenews.com.pk|upi.com|usatoday|washingtonpost|wsj.com|xinhuanet|bbj.hu|menafn.com|www.ft.com|seekingalpha.com|www.thefridaytimes.com|www.herald.ie|youtube|www.ign2.com|preen|theverge.com|phys.org|technology.inquirer|gizmodo.com|engadget.com|arstechnica|techhq|androidpolice|macrumors|www.wired.com|www.thurrott.com|seekingalpha|deadspin.com|aol.com|eonline.com|www.cnet.com|www.manilachannel.com|http://www.infotag.md|http://dailyexplorer.net|https://www.notebookcheck.net|https://www.androidauthority.com|https://www.phonearena.com|https://bleacherreport.com|https://www.vulture.com|http://admin.punchng.com|https://www.androidcentral.com|https

In [9]:
def save_chunks(idx):
    df1 = pd.read_json('data/x%03d'%idx, lines=True)
    df_save = np.array_split(df1, 10)
    
    new_index  = range(10*idx, 10*(idx+1))   
    for chunk, filename in enumerate(new_index):
         df_save[chunk].to_csv('split/%03d.csv'%filename, index=False)
    print('%s to %s'%(idx, new_index))
    
    
pandarallel.initialize(nb_workers=10, progress_bar=False) 
_= pd.Series(range(58)).parallel_apply(save_chunks) 

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
36 to range(360, 370)
48 to range(480, 490)
42 to range(420, 430)
6 to range(60, 70)
30 to range(300, 310)
24 to range(240, 250)
53 to range(530, 540)
0 to range(0, 10)
12 to range(120, 130)
18 to range(180, 190)
37 to range(370, 380)
43 to range(430, 440)
7 to range(70, 80)
49 to range(490, 500)
25 to range(250, 260)
31 to range(310, 320)
1 to range(10, 20)
54 to range(540, 550)
38 to range(380, 390)
13 to range(130, 140)
50 to range(500, 510)
44 to range(440, 450)
19 to range(190, 200)
32 to range(320, 330)
8 to range(80, 90)
2 to range(20, 30)
26 to range(260, 270)
55 to range(550, 560)
39 to range(390, 400)
51 to range(510, 520)
14 to range(140, 150)
45 to range(450, 460)
33 to range(330, 340)
20 to range(200, 210)
40 to range(400, 410)
27 to range(270, 280)
56 to range(560, 570)
3 to range(30, 40)
9 to range(90, 100)
21 to range(210, 220)
4

In [10]:
def filter_by_urls(idx):
    try:
        sizes=[]
        df1 = pd.read_csv('split/%03d.csv'%idx,header=0)
        sizes.append(df1.shape[0])


        df1 = df1.loc[:, ['content','source','date','title', 'url']]
        df1 = df1.rename(columns = {'content': 'text'})

        df1 = df1[~df1.duplicated(subset= 'url')]
        df1 = df1[~df1.url.str.contains(excude_domain)]
        df1 = df1[~df1.source.str.contains('int_the_news|thenews_pk')]
        df1 = df1[df1.source!='ap']

        df1 = df1[~df1.text.isnull()]    
        df1 = df1[df1.text.str.len()>200]
        df1 = df1[~df1.text.duplicated()].reset_index(drop=True)     


        df1['start'] = df1['text'].apply(lambda x: x[0:200])
        df1.loc[df1.title.isnull(),'title'] = '\n'
        df1.loc[df1.url.isnull(),'title'] = '\n'

        df1 = df1[~df1.duplicated(subset= ['title','start'])]
        df1 = df1[~df1.start.str.contains('subscribe|SUNDAY TARGET|access to the following|error|cannot be found|\b404\b|To view this|View more')]

        sizes.append(df1.shape[0])

        # ------------------- Start cleaning  --------------------------#

        df1.text = df1.text.apply(lambda x: html.unescape(x.replace('&amp;', '&')))
        df1.title = df1.title.apply(lambda x: html.unescape(x.replace('&amp;', '&')))
        df1.start = df1.start.apply(lambda x: html.unescape(x.replace('&amp;', '&')))

        df1.text = df1.text.apply(lambda x: unicodetoascii(x))
        df1.text = df1.text.apply(lambda x: unicodedata.normalize("NFKD", x))


        # email
        df1.text = df1.text.apply(lambda x: re.sub("\S+@\S+(?:\.\S+)+",'',x))

        # telphone
        df1.text = df1.text.apply(lambda x: re.sub('\(\+( |-|\d)+\)( |-|\d)+',' ',x))
        df1.text = df1.text.apply(lambda x: re.sub('\+( |-|\d)+',' ',x))

        # noise
        df1.text =\
        df1.text.apply(lambda x: re.sub('\n(ad|advertisement|tweet):?\n', "", x, flags=re.IGNORECASE))

        # urls
        df1.text = df1.text.apply(lambda x: re.sub(r"http\S+", "", x))

        # delete too many \n
        df1.text = df1.text.apply(lambda x: re.sub('\n\n+', "\n\n", x, flags=re.IGNORECASE))

        # head and tails
        df1.text = df1.text.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE)) 


        df1 = df1[df1.text.str.len()>200].reset_index(drop=True)
        #     print('End cleaning %s'%idx)
        # ------------------- Ending cleaning  --------------------------#



        # ------------------ Count Keywords ------------------#

        df1['title_in'] = df1['title'].apply(relevant_rules.findall)
        df1['title_out'] = df1['title'].apply(irelevant_rules.findall)

        df1['url_in'] = df1['url'].apply(relevant_rules.findall)
        df1['url_out'] = df1['url'].apply(irelevant_rules.findall)

        df1['text_in'] = df1['text'].apply(relevant_rules.findall)
        df1['text_out'] = df1['text'].apply(irelevant_rules.findall)


        df1['abs_in'] = df1['start'].apply(relevant_rules.findall)
        df1['abs_out'] = df1['start'].apply(irelevant_rules.findall)

        # ------------------ End counting Keywords ------------------#


        # ------------------  Threshold ------------------#

        relevant_count =\
            df1['title_in'].apply(lambda x:len(set(x.lower()))) +\
            df1['url_in'].apply(lambda x:len(set(x.lower()))) +\
            df1['abs_in'].apply(lambda x:len(set(x.lower())))

        irelevant_count =\
            df1['title_out'].apply(lambda x:len(set(x.lower()))) +\
            df1['url_out'].apply(lambda x:len(set(x.lower()))) +\
            df1['abs_out'].apply(lambda x:len(set(x.lower())))

        text_in_count = df1['text_in'].apply(lambda x:len(set(x.lower())))
        text_out_count = df1['text_out'].apply(lambda x:len(set(x.lower())))


        df1['rel_count'] = relevant_count
        df1['irel_count'] = irelevant_count
        df1['in_count'] = text_in_count
        df1['out_count'] = text_out_count

        remove_index =\
            ((relevant_count==0) & (irelevant_count > 0)) |\
            ((relevant_count >1) & (irelevant_count > 1)) |\
            ( text_in_count < 2) |\
            ((text_in_count >=2) & (text_in_count < text_out_count + 1))

        remove_index = df1[remove_index== True].index
        sizes.append(remove_index.shape[0])

        df1 = df1[~df1.index.isin(remove_index)]
        sizes.append(df1.shape[0])


        df1.to_csv('416/%03d.csv'%idx, index=False)
        print('%s ---- %s'%(idx, sizes))
    except:
        print('\n\t ---error:  %s'%idx)

In [14]:
num_chunk= 580
pandarallel.initialize(nb_workers=32)
df_save = pd.DataFrame([idx for idx in range(num_chunk)],columns = ['idx'])

_ = df_save.idx.parallel_apply(filter_by_urls)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
526 ---- [10000, 1589, 621, 966]
382 ---- [10000, 1546, 632, 913]
328 ---- [10000, 1570, 615, 951]
508 ---- [10000, 1561, 573, 988]
364 ---- [10000, 1676, 535, 1139]
19 ---- [10000, 1960, 664, 1295]
436 ---- [10000, 1789, 724, 1059]
400 ---- [10000, 1688, 680, 1007]
346 ---- [10000, 1764, 704, 1060]
38 ---- [10000, 2073, 736, 1336]
274 ---- [10000, 1997, 708, 1287]
292 ---- [10000, 1847, 694, 1153]
472 ---- [10000, 2178, 801, 1377]
130 ---- [10000, 2555, 879, 1676]
490 ---- [10000, 2305, 775, 1530]
76 ---- [10000, 2561, 912, 1648]
94 ---- [10000, 2779, 922, 1857]
238 ---- [10000, 2456, 814, 1641]
148 ---- [10000, 2615, 998, 1615]
454 ---- [10000, 2725, 1008, 1717]
166 ---- [10000, 2874, 1012, 1861]
256 ---- [10000, 2626, 936, 1690]
184 ---- [10000, 3046, 1075, 1966]
220 ---- [10000, 2916, 1044, 1872]
202 ---- [10000, 2999, 1037, 1962]
509 ---- [

318 ---- [10000, 4702, 1905, 2792]
36 ---- [10000, 2339, 825, 1513]
197 ---- [10000, 2897, 1036, 1861]
535 ---- [10000, 4571, 1663, 2908]
91 ---- [10000, 2415, 877, 1538]
469 ---- [10000, 2194, 859, 1335]
552 ---- [10000, 4273, 1641, 2632]
574 ---- [5187, 1516, 541, 975]
180 ---- [10000, 3175, 1164, 2011]
234 ---- [10000, 2688, 1040, 1644]
506 ---- [10000, 2069, 763, 1300]
452 ---- [10000, 3500, 1334, 2166]
16 ---- [10000, 2902, 1006, 1896]
124 ---- [10000, 2233, 773, 1460]
254 ---- [10000, 2441, 837, 1604]
291 ---- [10000, 2119, 733, 1385]
507 ---- [10000, 1598, 604, 993]
37 ---- [10000, 2185, 791, 1393]
73 ---- [10000, 2820, 1015, 1805]
108 ---- [10000, 3104, 1099, 2001]
216 ---- [10000, 2829, 1011, 1815]
453 ---- [10000, 2779, 1069, 1710]
235 ---- [10000, 2612, 909, 1701]
92 ---- [10000, 2859, 1117, 1741]
470 ---- [10000, 2119, 750, 1369]
55 ---- [10000, 3883, 1475, 2407]
575 ---- [5187, 1644, 617, 1027]
271 ---- [10000, 5123, 1639, 3484]
198 ---- [10000, 3053, 1083, 1966]
17 ---- [

# merge

In [17]:
num_chunk=580
pandarallel.initialize(progress_bar= True)
df_save = pd.DataFrame([idx for idx in range(num_chunk)],columns = ['idx'])
values  = df_save.idx.parallel_apply(lambda idx: pd.read_csv('416/%03d.csv'%idx,header=[0]))

df_all = pd.concat(values.values).reset_index(drop=True)
del values

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [18]:
# df_all = df_all.loc[:, ['content','source','date','title', 'url']]
# df_all = df_all.rename(columns = {'content': 'text'})
# df_all.start =
df_all

,text,source,date,title,url,start,title_in,title_out,url_in,url_out,text_in,text_out,abs_in,abs_out,rel_count,irel_count,in_count,out_count
0,"New Delhi: Calling for attacking corruption at the roots, Vice President M Venkaiah Naidu on Monday asked civil servants not to be misled by ""black sheep"" and render their service without fear or favour. Naidu also hailed GST as a revolutionary step and praised demonetisation. ""As you are new generation of civil servants, I hope you will not be misled by black sheep in the fold but would render your service without fear or favour and without any expectation of extraneous rewards. If you serve in the true spirit of service I am sure you will have your best reward,"" he said while inaugurat...",india_zee_states,"Monday, October 30, 2017, 20:22 +0530",Don't be misled by black sheep: Venkaiah Naidu tells civil servants,http://zeenews.india.com/delhi/dont-be-misled-by-black-sheep-venkaiah-naidu-tells-civil-servants-2053007.html,"New Delhi: Calling for attacking corruption at the roots, Vice President M Venkaiah Naidu on Monday asked civil servants not to be misled by ""black sheep"" and render their service without fear or favo","['civil', 'servants']",[],"['civil', 'servants']",[],"['attack', 'corrupt', 'President', 'civil', 'servants', 'revolution', 'civil', 'servants', 'Vigilance', 'Vigilance', 'corrupt', 'laws', 'corrupt', 'civil', 'servants', 'reform', 'Minister', 'reform', 'corrupt', 'strike', 'vigilance', 'Vigilance', 'Corrupt', 'President', 'govern', 'welfare', 'Vigilance', 'President', 'civil', 'govern', 'reform']",['education'],"['attack', 'corrupt', 'President', 'civil', 'servants']",[],9,0,15,1
1,"The UK's minister for agriculture has said he is hopeful that Brexit negotiations will produce a deal that is in everybody's interests. Michael Gove indicated that he plans to travel to Dublin next month for talks with members of the Government and with representatives of the Irish agricultural sector. He was speaking to RTÉ News following a meeting with the EU's Agriculture Commissioner, Phil Hogan, on the sidelines of a conference on the future of the dairy sector. Asked if the UK might be pursuing a treaty with the European Union that would give them a version of the current customs...",ireland_rte,2017-10-30T15:48:48UTC,Gove and Hogan discuss Brexit at dairy conference,http://www.rte.ie/news/2017/1030/916224-brexit/,The UK's minister for agriculture has said he is hopeful that Brexit negotiations will produce a deal that is in everybody's interests. Michael Gove indicated that he plans to travel to Dublin next m,[],[],[],[],"['minister', 'Govern', 'Minister', 'minister', 'border', 'minister']","['deal', 'travel', 'deal', 'trade']",['minister'],"['deal', 'travel']",1,2,4,3
2,"By EUOBSERVER Northern Ireland is heading towards an intervention from London after talks between Irish nationalist Sinn Fein and the pro-British DUP ended Friday without an agreement. Northern Ireland has been without a regional administration since January. London set Monday as a deadline for an agreement, to avoid a budget being introduced from Westminster. Northern Ireland state secretary James Brokenshire warned of ""a glide path to greater and greater UK government intervention"".",euroobs,"Mon, 30 Oct 2017 08:58:55 +0100",[Ticker] Deadline looms for Northern Ireland negotiations,https://euobserver.com/tickers/139693?utm_medium=rss,By EUOBSERVER Northern Ireland is heading towards an intervention from London after talks between Irish nationalist Sinn Fein and the pro-British DUP ended Friday without an agreement. Northern Irela,[],[],[],[],"['nationalist', 'administration', 'govern']",[],['nationalist'],[],1,0,3,0
3,"Rohingya Refugee Women Bring Stories of Unspeakable Violence COX'S BAZAR, Bangladesh, Oct 10 2017 (IPS) - Yasmin, 26, holds her 10-day-old baby, who she gave birth to in a crowded refugee camp in Cox’s Bazar, a southeastern district bordering Myanmar. Three weeks ago, when she was still in

In [19]:
print(df_all.shape)
df_all = df_all[~df_all.duplicated(subset= 'text')]
print(df_all.shape)
df_all = df_all[~df_all.duplicated(subset= 'url')]
print(df_all.shape)
df_all = df_all[~df_all.duplicated(subset= ['title','start'])]
print(df_all.shape)

(989194, 18)
(984208, 18)
(984208, 18)
(982370, 18)


In [ ]:
df_all.to_csv('df_all_416.csv', index=False)

In [16]:
# num_chunks = 100
# df_save = np.array_split(df_all, num_chunks)
# save_chunks = lambda x: df_save[x].to_csv('step2/%03d.csv'%x, index=False)
# pandarallel.initialize(progress_bar=True)       
# _ = pd.Series(range(num_chunks)).parallel_apply(save_chunks) 

# Filter URLs

In [20]:
# df_all = pd.read_csv('df_all_416.csv', header=0)
df_all = df_all[~df_all.url.str.contains('sport|film|market|finance|business|invest')].reset_index(drop=True)
df_all

,text,source,date,title,url,start,title_in,title_out,url_in,url_out,text_in,text_out,abs_in,abs_out,rel_count,irel_count,in_count,out_count
0,"New Delhi: Calling for attacking corruption at the roots, Vice President M Venkaiah Naidu on Monday asked civil servants not to be misled by ""black sheep"" and render their service without fear or favour. Naidu also hailed GST as a revolutionary step and praised demonetisation. ""As you are new generation of civil servants, I hope you will not be misled by black sheep in the fold but would render your service without fear or favour and without any expectation of extraneous rewards. If you serve in the true spirit of service I am sure you will have your best reward,"" he said while inaugurat...",india_zee_states,"Monday, October 30, 2017, 20:22 +0530",Don't be misled by black sheep: Venkaiah Naidu tells civil servants,http://zeenews.india.com/delhi/dont-be-misled-by-black-sheep-venkaiah-naidu-tells-civil-servants-2053007.html,"New Delhi: Calling for attacking corruption at the roots, Vice President M Venkaiah Naidu on Monday asked civil servants not to be misled by ""black sheep"" and render their service without fear or favo","['civil', 'servants']",[],"['civil', 'servants']",[],"['attack', 'corrupt', 'President', 'civil', 'servants', 'revolution', 'civil', 'servants', 'Vigilance', 'Vigilance', 'corrupt', 'laws', 'corrupt', 'civil', 'servants', 'reform', 'Minister', 'reform', 'corrupt', 'strike', 'vigilance', 'Vigilance', 'Corrupt', 'President', 'govern', 'welfare', 'Vigilance', 'President', 'civil', 'govern', 'reform']",['education'],"['attack', 'corrupt', 'President', 'civil', 'servants']",[],9,0,15,1
1,"The UK's minister for agriculture has said he is hopeful that Brexit negotiations will produce a deal that is in everybody's interests. Michael Gove indicated that he plans to travel to Dublin next month for talks with members of the Government and with representatives of the Irish agricultural sector. He was speaking to RTÉ News following a meeting with the EU's Agriculture Commissioner, Phil Hogan, on the sidelines of a conference on the future of the dairy sector. Asked if the UK might be pursuing a treaty with the European Union that would give them a version of the current customs...",ireland_rte,2017-10-30T15:48:48UTC,Gove and Hogan discuss Brexit at dairy conference,http://www.rte.ie/news/2017/1030/916224-brexit/,The UK's minister for agriculture has said he is hopeful that Brexit negotiations will produce a deal that is in everybody's interests. Michael Gove indicated that he plans to travel to Dublin next m,[],[],[],[],"['minister', 'Govern', 'Minister', 'minister', 'border', 'minister']","['deal', 'travel', 'deal', 'trade']",['minister'],"['deal', 'travel']",1,2,4,3
2,"By EUOBSERVER Northern Ireland is heading towards an intervention from London after talks between Irish nationalist Sinn Fein and the pro-British DUP ended Friday without an agreement. Northern Ireland has been without a regional administration since January. London set Monday as a deadline for an agreement, to avoid a budget being introduced from Westminster. Northern Ireland state secretary James Brokenshire warned of ""a glide path to greater and greater UK government intervention"".",euroobs,"Mon, 30 Oct 2017 08:58:55 +0100",[Ticker] Deadline looms for Northern Ireland negotiations,https://euobserver.com/tickers/139693?utm_medium=rss,By EUOBSERVER Northern Ireland is heading towards an intervention from London after talks between Irish nationalist Sinn Fein and the pro-British DUP ended Friday without an agreement. Northern Irela,[],[],[],[],"['nationalist', 'administration', 'govern']",[],['nationalist'],[],1,0,3,0
3,"Rohingya Refugee Women Bring Stories of Unspeakable Violence COX'S BAZAR, Bangladesh, Oct 10 2017 (IPS) - Yasmin, 26, holds her 10-day-old baby, who she gave birth to in a crowded refugee camp in Cox’s Bazar, a southeastern district bordering Myanmar. Three weeks ago, when she was still in

In [21]:
url_count = df_all.source.value_counts()
for i in url_count.keys():
    print(i, '\t\t',url_count[i])

hindu_nat 		 59736
hindu_cities 		 58752
minnstartrib_national 		 44953
phil_inquirer 		 37032
nigeria_dailypost 		 36488
nigeria_thepunch 		 33000
ocregister_ap 		 30303
minnstartrib_world 		 29649
google 		 27680
kenya_nation 		 26205
southaf_iol 		 21912
nzherald_world 		 21218
toronto_star_world 		 20784
nigeria_sunnews 		 19118
india_deccanher_news 		 17419
uk_telegraph_politics 		 14941
dawn_pk 		 14070
nigeria_tidenews 		 13493
jpost_front 		 13158
india_zee_states 		 12471
daily_monitor_uganda 		 12154
philstar_headlines 		 11919
ireland_rte 		 11892
nigeria_pilot 		 11754
australia_smh_world 		 11296
hurriyet 		 10930
india_zee_national 		 10433
hindu_int 		 10265
thailand_bankokpost 		 10030
nigeria_saharareporters 		 9552
india_mint_econpol 		 9363
nxherald_national 		 9043
houstoncron_news 		 7986
nigeria_promptnews 		 7957
phil_manilastandard 		 7602
euronews 		 7549
southaf_capetownt 		 7361
nigeria_blueprint 		 7340
malta_today 		 7252
almonitor 		 7049
canada_globalnews

In [23]:
website = df_all.url.apply(lambda x: x.replace('http://','').replace('https://','').split('/')[0])
website.value_counts()
top130 = website.value_counts()[0:130]
top130 = list(top130.keys())
# top110

In [24]:
website.value_counts()[100:130]

www.yahoo.com                             60
www.independent.co.uk                     59
www.ksl.com                               53
www.newser.com                            53
www.egyptindependent.com                  50
komonews.com                              44
english.kyodonews.jp                      43
news.punchng.com                          40
beta.philstar.com                         36
www.chicagotribune.com                    35
www.bostonglobe.com                       30
www.buenosairesherald.com                 27
www.huffpost.com                          26
www.ign.com                               26
news.yahoo.com                            26
www.sciencealert.com                      25
www.xxlmag.com                            24
english.hankyung.com                      24
www.thesun.co.uk                          23
www.arabnews.com                          23
www.thisisinsider.com                     22
www.statnews.com                          21
www.cheats

In [25]:
df_all.loc[df_all.source=='google','url'].apply(lambda x: x.replace('http://','').replace('https://','').replace('news.google.com/','').split('/')[0]).value_counts().head(50)

news                      12447
__i                        8429
www.foxnews.com            2363
www.cbsnews.com             616
www.vox.com                 405
www.huffingtonpost.com      241
www.thedailybeast.com       204
slate.com                   186
www.newsweek.com            156
time.com                    131
www.dailymail.co.uk         129
www.salon.com                75
www.yahoo.com                60
www.independent.co.uk        59
www.newser.com               53
www.ksl.com                  53
komonews.com                 44
www.chicagotribune.com       35
www.bostonglobe.com          30
www.ign.com                  26
news.yahoo.com               26
www.huffpost.com             26
www.sciencealert.com         25
www.xxlmag.com               24
www.thesun.co.uk             23
www.thisisinsider.com        22
www.cheatsheet.com           21
www.statnews.com             21
www.wcvb.com                 19
www.miamiherald.com          18
www.rollingstone.com         17
jalopnik

In [26]:
keywords4= '|'.join(top130)
keywords4 = re.compile(keywords4, re.IGNORECASE)
keywords4

re.compile(r'www.thehindu.com|www.startribune.com|www.iol.co.za|dailypost.ng|www.thestar.com|newsinfo.inquirer.net|punchng.com|hosted2.ap.org|www.nzherald.co.nz|zeenews.india.com|www.nation.co.ke|feedproxy.google.com|news.google.com|www.philstar.com|sunnewsonline.com|www.deccanherald.com|www.smh.com.au|www.telegraph.co.uk|www.livemint.com|www.bangkokpost.com|www.jpost.com|www.thetidenewsonline.com|www.monitor.co.ug|www.rte.ie|nigerianpilot.com|www.taipeitimes.com|www.thestar.com.my|www.hurriyetdailynews.com|www.dailystar.com.lb|saharareporters.com|www.chron.com|www.promptnewsonline.com|www.maltatoday.com.mt|www.al-monitor.com|globalnews.ca|www.vanguardngr.com|english.alarabiya.net|euobserver.com|www.middleeasteye.net|english.chosun.com|www.cbsnews.com|cyprus-mail.com|www.theeastafrican.co.ke|www.osundefender.com|voiceofvienna.org|manilastandardtoday.com|leadership.ng|www.blueprint.ng|en.mercopress.com|blueprint.ng|thefrontierpost.com|www.the-star.co.ke|independent.ng|www.nineoclock.ro|

In [27]:
keywords5 =\
'http://www.infotag.md|'+\
'http://dailyexplorer.net|'+\
'https://www.notebookcheck.net|'+\
'https://www.androidauthority.com|'+\
'https://www.phonearena.com|'+\
'https://bleacherreport.com|'+\
'https://www.vulture.com|'+\
'http://admin.punchng.com|'+\
'https://www.androidcentral.com|'+\
'https://www.livescience.com|'+\
'www.espn.com|'+\
'seinstrukcii.date'+\
'rollingstone'+\
'www.cheatsheet.com'+\
'www.polygon.com'+\
'hollywoodreporter'+\
'theathletic.com'

keywords5 = re.compile(keywords5, re.IGNORECASE)
keywords5

re.compile(r'http://www.infotag.md|http://dailyexplorer.net|https://www.notebookcheck.net|https://www.androidauthority.com|https://www.phonearena.com|https://bleacherreport.com|https://www.vulture.com|http://admin.punchng.com|https://www.androidcentral.com|https://www.livescience.com|www.espn.com|seinstrukcii.daterollingstonewww.cheatsheet.comwww.polygon.comhollywoodreportertheathletic.com',
re.IGNORECASE|re.UNICODE)

In [28]:
# include keywords4,  excude keywords5

df1 = df_all
print(df1.shape)
df1 = df1[df1.url.str.contains(keywords4)]
df1 = df1[~df1.url.str.contains(keywords5)]
print(df1.shape)

(967597, 18)
(965802, 18)


In [5]:
# def second_process(filename):
#     df1 = pd.read_csv(filename, header=[0])
#     if 'title_in' in df1.columns:
#         return
#     df1['title_in'] = df1['title'].apply(relevant_rules.findall)
#     df1['title_out'] = df1['title'].apply(irelevant_rules.findall)

#     df1['url_in'] = df1['url'].apply(relevant_rules.findall)
#     df1['url_out'] = df1['url'].apply(irelevant_rules.findall)

#     df1['abs_in'] = df1['start'].apply(relevant_rules.findall)
#     df1['abs_out'] = df1['start'].apply(irelevant_rules.findall)

#     df1['text_in'] = df1['text'].apply(relevant_rules.findall)
#     df1['text_out'] = df1['text'].apply(irelevant_rules.findall)
#     df1.to_csv(filename, index=False)
#     print(filename)
    
    
# pandarallel.initialize(nb_workers =16, progress_bar=False)    
# _ = df['path'].parallel_apply(second_process)   

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
416_step2/083.csv
416_step2/065.csv
416_step2/077.csv
416_step2/089.csv
416_step2/029.csv
416_step2/007.csv
416_step2/035.csv
416_step2/021.csv
416_step2/059.csv
416_step2/095.csv
416_step2/014.csv
416_step2/071.csv
416_step2/053.csv
416_step2/041.csv
416_step2/047.csv
416_step2/001.csv
416_step2/084.csv
416_step2/066.csv
416_step2/090.csv
416_step2/078.csv
416_step2/030.csv
416_step2/072.csv
416_step2/096.csv
416_step2/023.csv
416_step2/015.csv
416_step2/036.csv
416_step2/060.csv
416_step2/009.csv
416_step2/054.csv
416_step2/048.csv
416_step2/042.csv
416_step2/002.csv
416_step2/086.csv
416_step2/080.csv
416_step2/068.csv
416_step2/092.csv
416_step2/032.csv
416_step2/062.csv
416_step2/074.csv
416_step2/017.csv
416_step2/038.csv
416_step2/010.csv
416_step2/056.csv
416_step2/024.csv
416_step2/098.csv
416_step2/050.csv
416_step2/044.csv
416_step2/0

In [90]:
df1.to_csv('df_all_417.csv', index=False)

# Filtering

In [3]:
# df1= pd.read_csv('df_all_417.csv', header=0)
df1 = df1.drop(columns='start')
df1

,text,source,date,title,url,title_in,title_out,url_in,url_out,text_in,text_out,abs_in,abs_out,rel_count,irel_count,in_count,out_count
0,"New Delhi: Calling for attacking corruption at the roots, Vice President M Venkaiah Naidu on Monday asked civil servants not to be misled by ""black sheep"" and render their service without fear or favour. Naidu also hailed GST as a revolutionary step and praised demonetisation. ""As you are new generation of civil servants, I hope you will not be misled by black sheep in the fold but would render your service without fear or favour and without any expectation of extraneous rewards. If you serve in the true spirit of service I am sure you will have your best reward,"" he said while inaugurat...",india_zee_states,"Monday, October 30, 2017, 20:22 +0530",Don't be misled by black sheep: Venkaiah Naidu tells civil servants,http://zeenews.india.com/delhi/dont-be-misled-by-black-sheep-venkaiah-naidu-tells-civil-servants-2053007.html,"['civil', 'servants']",[],"['civil', 'servants']",[],"['attack', 'corrupt', 'President', 'civil', 'servants', 'revolution', 'civil', 'servants', 'Vigilance', 'Vigilance', 'corrupt', 'laws', 'corrupt', 'civil', 'servants', 'reform', 'Minister', 'reform', 'corrupt', 'strike', 'vigilance', 'Vigilance', 'Corrupt', 'President', 'govern', 'welfare', 'Vigilance', 'President', 'civil', 'govern', 'reform']",['education'],"['attack', 'corrupt', 'President', 'civil', 'servants']",[],5,0,13,1
1,"The UK's minister for agriculture has said he is hopeful that Brexit negotiations will produce a deal that is in everybody's interests. Michael Gove indicated that he plans to travel to Dublin next month for talks with members of the Government and with representatives of the Irish agricultural sector. He was speaking to RTÉ News following a meeting with the EU's Agriculture Commissioner, Phil Hogan, on the sidelines of a conference on the future of the dairy sector. Asked if the UK might be pursuing a treaty with the European Union that would give them a version of the current customs...",ireland_rte,2017-10-30T15:48:48UTC,Gove and Hogan discuss Brexit at dairy conference,http://www.rte.ie/news/2017/1030/916224-brexit/,[],[],[],[],"['minister', 'Govern', 'Minister', 'minister', 'border', 'minister']","['deal', 'travel', 'deal', 'trade']",['minister'],"['deal', 'travel']",1,2,3,3
2,"By EUOBSERVER Northern Ireland is heading towards an intervention from London after talks between Irish nationalist Sinn Fein and the pro-British DUP ended Friday without an agreement. Northern Ireland has been without a regional administration since January. London set Monday as a deadline for an agreement, to avoid a budget being introduced from Westminster. Northern Ireland state secretary James Brokenshire warned of ""a glide path to greater and greater UK government intervention"".",euroobs,"Mon, 30 Oct 2017 08:58:55 +0100",[Ticker] Deadline looms for Northern Ireland negotiations,https://euobserver.com/tickers/139693?utm_medium=rss,[],[],[],[],"['nationalist', 'administration', 'govern']",[],['nationalist'],[],1,0,3,0
3,"Rohingya Refugee Women Bring Stories of Unspeakable Violence COX'S BAZAR, Bangladesh, Oct 10 2017 (IPS) - Yasmin, 26, holds her 10-day-old baby, who she gave birth to in a crowded refugee camp in Cox’s Bazar, a southeastern district bordering Myanmar. Three weeks ago, when she was still in her home in Hpaung Taw Pyin village in Myanmar, she was raped by a group of soldiers as houses burned, people fled and gunfire shattered the air. “I have been working as a human rights activist for the last 20 years but never heard of such an extreme level of violence."" --Bimol Chandra Dey Sarker, Chi...",ips_gender,2017-10-10T12:26:28UTC,Rohingya Refugee Women Bring Stories of Unspeakable Violence,http://www.ipsnews.net/2017/10/rohingya-refugee-women-bring-stories-unspeakable-violence/?utm_source=rss&utm_medium=rss&utm_campaign=rohingya-refugee-women-bring-stories-unspeakable-violence,"['Refuge', 'Vio

In [19]:
df1.title_in[0]

"['civil', 'servants']"

In [4]:
df1.title_in = df1.title_in.apply(eval).apply(Counter).apply(dict)
df1.title_out = df1.title_out.apply(eval).apply(Counter).apply(dict)
df1.url_in = df1.url_in.apply(eval).apply(Counter).apply(dict)
df1.url_out = df1.url_out.apply(eval).apply(Counter).apply(dict)
df1.text_in = df1.text_in.apply(eval).apply(Counter).apply(dict)
df1.text_out = df1.text_out.apply(eval).apply(Counter).apply(dict)
df1.abs_in = df1.abs_in.apply(eval).apply(Counter).apply(dict)
df1.abs_out = df1.abs_out.apply(eval).apply(Counter).apply(dict)

# Save to two copies

In [11]:
# df1.to_csv('large_417.csv', index=False)
df1

,text,source,date,title,url,title_in,title_out,url_in,url_out,text_in,text_out,abs_in,abs_out,rel_count,irel_count,in_count,out_count
0,"New Delhi: Calling for attacking corruption at the roots, Vice President M Venkaiah Naidu on Monday asked civil servants not to be misled by ""black sheep"" and render their service without fear or favour. Naidu also hailed GST as a revolutionary step and praised demonetisation. ""As you are new generation of civil servants, I hope you will not be misled by black sheep in the fold but would render your service without fear or favour and without any expectation of extraneous rewards. If you serve in the true spirit of service I am sure you will have your best reward,"" he said while inaugurat...",india_zee_states,"Monday, October 30, 2017, 20:22 +0530",Don't be misled by black sheep: Venkaiah Naidu tells civil servants,http://zeenews.india.com/delhi/dont-be-misled-by-black-sheep-venkaiah-naidu-tells-civil-servants-2053007.html,"{'civil': 1, 'servants': 1}",{},"{'civil': 1, 'servants': 1}",{},"{'attack': 1, 'corrupt': 4, 'President': 3, 'civil': 4, 'servants': 3, 'revolution': 1, 'Vigilance': 4, 'laws': 1, 'reform': 3, 'Minister': 1, 'strike': 1, 'vigilance': 1, 'Corrupt': 1, 'govern': 2, 'welfare': 1}",{'education': 1},"{'attack': 1, 'corrupt': 1, 'President': 1, 'civil': 1, 'servants': 1}",{},5,0,13,1
1,"The UK's minister for agriculture has said he is hopeful that Brexit negotiations will produce a deal that is in everybody's interests. Michael Gove indicated that he plans to travel to Dublin next month for talks with members of the Government and with representatives of the Irish agricultural sector. He was speaking to RTÉ News following a meeting with the EU's Agriculture Commissioner, Phil Hogan, on the sidelines of a conference on the future of the dairy sector. Asked if the UK might be pursuing a treaty with the European Union that would give them a version of the current customs...",ireland_rte,2017-10-30T15:48:48UTC,Gove and Hogan discuss Brexit at dairy conference,http://www.rte.ie/news/2017/1030/916224-brexit/,{},{},{},{},"{'minister': 3, 'Govern': 1, 'Minister': 1, 'border': 1}","{'deal': 2, 'travel': 1, 'trade': 1}",{'minister': 1},"{'deal': 1, 'travel': 1}",1,2,3,3
2,"By EUOBSERVER Northern Ireland is heading towards an intervention from London after talks between Irish nationalist Sinn Fein and the pro-British DUP ended Friday without an agreement. Northern Ireland has been without a regional administration since January. London set Monday as a deadline for an agreement, to avoid a budget being introduced from Westminster. Northern Ireland state secretary James Brokenshire warned of ""a glide path to greater and greater UK government intervention"".",euroobs,"Mon, 30 Oct 2017 08:58:55 +0100",[Ticker] Deadline looms for Northern Ireland negotiations,https://euobserver.com/tickers/139693?utm_medium=rss,{},{},{},{},"{'nationalist': 1, 'administration': 1, 'govern': 1}",{},{'nationalist': 1},{},1,0,3,0
3,"Rohingya Refugee Women Bring Stories of Unspeakable Violence COX'S BAZAR, Bangladesh, Oct 10 2017 (IPS) - Yasmin, 26, holds her 10-day-old baby, who she gave birth to in a crowded refugee camp in Cox’s Bazar, a southeastern district bordering Myanmar. Three weeks ago, when she was still in her home in Hpaung Taw Pyin village in Myanmar, she was raped by a group of soldiers as houses burned, people fled and gunfire shattered the air. “I have been working as a human rights activist for the last 20 years but never heard of such an extreme level of violence."" --Bimol Chandra Dey Sarker, Chi...",ips_gender,2017-10-10T12:26:28UTC,Rohingya Refugee Women Bring Stories of Unspeakable Violence,http://www.ipsnews.net/2017/10/rohingya-refugee-women-bring-stories-unspeakable-violence/?utm_source=rss&utm_medium=rss&utm_campaign=rohingya-refugee-women-bring-stories-unspeakable-violence,"{'Refuge': 1, 'Violen': 1}",{},"{'refuge': 2, 'violen': 2}",{},"{'Refuge': 2, 'Violen': 1, 'refuge': 10, 'bor

In [12]:
num_chunks = 200

df_save = np.array_split(df1, num_chunks)
save_chunks = lambda x: df_save[x].to_csv('step3/%03d.csv'%x, index=False)
pandarallel.initialize(progress_bar=True)       
_ = pd.Series(range(num_chunks)).parallel_apply(save_chunks) 

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


#  Fix bugs in 417

In [3]:
df1= pd.read_csv('large_417.csv', header=0)
df1

,text,source,date,title,url,title_in,title_out,url_in,url_out,text_in,text_out,abs_in,abs_out,rel_count,irel_count,in_count,out_count
0,"New Delhi: Calling for attacking corruption at the roots, Vice President M Venkaiah Naidu on Monday asked civil servants not to be misled by ""black sheep"" and render their service without fear or favour. Naidu also hailed GST as a revolutionary step and praised demonetisation. ""As you are new generation of civil servants, I hope you will not be misled by black sheep in the fold but would render your service without fear or favour and without any expectation of extraneous rewards. If you serve in the true spirit of service I am sure you will have your best reward,"" he said while inaugurat...",india_zee_states,"Monday, October 30, 2017, 20:22 +0530",Don't be misled by black sheep: Venkaiah Naidu tells civil servants,http://zeenews.india.com/delhi/dont-be-misled-by-black-sheep-venkaiah-naidu-tells-civil-servants-2053007.html,"{'civil': 1, 'servants': 1}",{},"{'civil': 1, 'servants': 1}",{},"{'attack': 1, 'corrupt': 4, 'President': 3, 'civil': 4, 'servants': 3, 'revolution': 1, 'Vigilance': 4, 'laws': 1, 'reform': 3, 'Minister': 1, 'strike': 1, 'vigilance': 1, 'Corrupt': 1, 'govern': 2, 'welfare': 1}",{'education': 1},"{'attack': 1, 'corrupt': 1, 'President': 1, 'civil': 1, 'servants': 1}",{},5,0,13,1
1,"The UK's minister for agriculture has said he is hopeful that Brexit negotiations will produce a deal that is in everybody's interests. Michael Gove indicated that he plans to travel to Dublin next month for talks with members of the Government and with representatives of the Irish agricultural sector. He was speaking to RTÉ News following a meeting with the EU's Agriculture Commissioner, Phil Hogan, on the sidelines of a conference on the future of the dairy sector. Asked if the UK might be pursuing a treaty with the European Union that would give them a version of the current customs...",ireland_rte,2017-10-30T15:48:48UTC,Gove and Hogan discuss Brexit at dairy conference,http://www.rte.ie/news/2017/1030/916224-brexit/,{},{},{},{},"{'minister': 3, 'Govern': 1, 'Minister': 1, 'border': 1}","{'deal': 2, 'travel': 1, 'trade': 1}",{'minister': 1},"{'deal': 1, 'travel': 1}",1,2,3,3
2,"By EUOBSERVER Northern Ireland is heading towards an intervention from London after talks between Irish nationalist Sinn Fein and the pro-British DUP ended Friday without an agreement. Northern Ireland has been without a regional administration since January. London set Monday as a deadline for an agreement, to avoid a budget being introduced from Westminster. Northern Ireland state secretary James Brokenshire warned of ""a glide path to greater and greater UK government intervention"".",euroobs,"Mon, 30 Oct 2017 08:58:55 +0100",[Ticker] Deadline looms for Northern Ireland negotiations,https://euobserver.com/tickers/139693?utm_medium=rss,{},{},{},{},"{'nationalist': 1, 'administration': 1, 'govern': 1}",{},{'nationalist': 1},{},1,0,3,0
3,"Rohingya Refugee Women Bring Stories of Unspeakable Violence COX'S BAZAR, Bangladesh, Oct 10 2017 (IPS) - Yasmin, 26, holds her 10-day-old baby, who she gave birth to in a crowded refugee camp in Cox’s Bazar, a southeastern district bordering Myanmar. Three weeks ago, when she was still in her home in Hpaung Taw Pyin village in Myanmar, she was raped by a group of soldiers as houses burned, people fled and gunfire shattered the air. “I have been working as a human rights activist for the last 20 years but never heard of such an extreme level of violence."" --Bimol Chandra Dey Sarker, Chi...",ips_gender,2017-10-10T12:26:28UTC,Rohingya Refugee Women Bring Stories of Unspeakable Violence,http://www.ipsnews.net/2017/10/rohingya-refugee-women-bring-stories-unspeakable-violence/?utm_source=rss&utm_medium=rss&utm_campaign=rohingya-refugee-women-bring-stories-unspeakable-violence,"{'Refuge': 1, 'Violen': 1}",{},"{'refuge': 2, 'violen': 2}",{},"{'Refuge': 2, 'Violen': 1, 'refuge': 10, 'bor

In [9]:
def update_counter(d):
    if type(d) == str:
        d=eval(d)
    c = Counter()
    for k,v in d.items():
        c.update({k.lower(): v})
    return c

# pandarallel.initialize(nb_workers=16, progress_bar=False)       


df1.title_in = df1.title_in.apply(update_counter)
df1.title_out = df1.title_out.apply(update_counter)
df1.url_in = df1.url_in.apply(update_counter)
df1.url_out = df1.url_out.apply(update_counter)
df1.abs_in = df1.abs_in.apply(update_counter)
df1.abs_out = df1.abs_out.apply(update_counter)
df1.text_in = df1.text_in.apply(update_counter)
df1.text_out = df1.text_out.apply(update_counter)

df1.rel_count = df1.title_in.str.len() + df1.url_in.str.len() + df1.abs_in.str.len()
df1.irel_count = df1.title_out.str.len() + df1.url_out.str.len() + df1.abs_out.str.len()
df1.in_count = df1.text_in.str.len()
df1.out_count = df1.text_out.str.len()

df1.title_in = df1.title_in.apply(dict)
df1.title_out = df1.title_out.apply(dict)
df1.url_in = df1.url_in.apply(dict)
df1.url_out = df1.url_out.apply(dict)
df1.abs_in = df1.abs_in.apply(dict)
df1.abs_out = df1.abs_out.apply(dict)
df1.text_in = df1.text_in.apply(dict)
df1.text_out = df1.text_out.apply(dict)

df1.to_csv('large_419.csv', index= False)

# Explore thresholds

In [2]:
df1 = pd.read_csv('large_419.csv', header=0)
df1

,text,source,date,title,url,title_in,title_out,url_in,url_out,text_in,text_out,abs_in,abs_out,rel_count,irel_count,in_count,out_count
0,"New Delhi: Calling for attacking corruption at the roots, Vice President M Venkaiah Naidu on Monday asked civil servants not to be misled by ""black sheep"" and render their service without fear or favour. Naidu also hailed GST as a revolutionary step and praised demonetisation. ""As you are new generation of civil servants, I hope you will not be misled by black sheep in the fold but would render your service without fear or favour and without any expectation of extraneous rewards. If you serve in the true spirit of service I am sure you will have your best reward,"" he said while inaugurat...",india_zee_states,"Monday, October 30, 2017, 20:22 +0530",Don't be misled by black sheep: Venkaiah Naidu tells civil servants,http://zeenews.india.com/delhi/dont-be-misled-by-black-sheep-venkaiah-naidu-tells-civil-servants-2053007.html,"{'civil': 1, 'servants': 1}",{},"{'civil': 1, 'servants': 1}",{},"{'attack': 1, 'corrupt': 5, 'president': 3, 'civil': 4, 'servants': 3, 'revolution': 1, 'vigilance': 5, 'laws': 1, 'reform': 3, 'minister': 1, 'strike': 1, 'govern': 2, 'welfare': 1}",{'education': 1},"{'attack': 1, 'corrupt': 1, 'president': 1, 'civil': 1, 'servants': 1}",{},9,0,13,1
1,"The UK's minister for agriculture has said he is hopeful that Brexit negotiations will produce a deal that is in everybody's interests. Michael Gove indicated that he plans to travel to Dublin next month for talks with members of the Government and with representatives of the Irish agricultural sector. He was speaking to RTÉ News following a meeting with the EU's Agriculture Commissioner, Phil Hogan, on the sidelines of a conference on the future of the dairy sector. Asked if the UK might be pursuing a treaty with the European Union that would give them a version of the current customs...",ireland_rte,2017-10-30T15:48:48UTC,Gove and Hogan discuss Brexit at dairy conference,http://www.rte.ie/news/2017/1030/916224-brexit/,{},{},{},{},"{'minister': 4, 'govern': 1, 'border': 1}","{'deal': 2, 'travel': 1, 'trade': 1}",{'minister': 1},"{'deal': 1, 'travel': 1}",1,2,3,3
2,"By EUOBSERVER Northern Ireland is heading towards an intervention from London after talks between Irish nationalist Sinn Fein and the pro-British DUP ended Friday without an agreement. Northern Ireland has been without a regional administration since January. London set Monday as a deadline for an agreement, to avoid a budget being introduced from Westminster. Northern Ireland state secretary James Brokenshire warned of ""a glide path to greater and greater UK government intervention"".",euroobs,"Mon, 30 Oct 2017 08:58:55 +0100",[Ticker] Deadline looms for Northern Ireland negotiations,https://euobserver.com/tickers/139693?utm_medium=rss,{},{},{},{},"{'nationalist': 1, 'administration': 1, 'govern': 1}",{},{'nationalist': 1},{},1,0,3,0
3,"Rohingya Refugee Women Bring Stories of Unspeakable Violence COX'S BAZAR, Bangladesh, Oct 10 2017 (IPS) - Yasmin, 26, holds her 10-day-old baby, who she gave birth to in a crowded refugee camp in Cox’s Bazar, a southeastern district bordering Myanmar. Three weeks ago, when she was still in her home in Hpaung Taw Pyin village in Myanmar, she was raped by a group of soldiers as houses burned, people fled and gunfire shattered the air. “I have been working as a human rights activist for the last 20 years but never heard of such an extreme level of violence."" --Bimol Chandra Dey Sarker, Chi...",ips_gender,2017-10-10T12:26:28UTC,Rohingya Refugee Women Bring Stories of Unspeakable Violence,http://www.ipsnews.net/2017/10/rohingya-refugee-women-bring-stories-unspeakable-violence/?utm_source=rss&utm_medium=rss&utm_campaign=rohingya-refugee-women-bring-stories-unspeakable-violence,"{'refuge': 1, 'violen': 1}",{},"{'refuge': 2, 'violen': 2}",{},"{'refuge': 12, 'violen': 7, 'border': 5, 'rape': 14, 'soldier': 8, 'human rights': 3, 'act

In [3]:
df1 = df1.iloc[:,[0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,4]]
df1

,text,source,date,title,title_in,title_out,url_in,url_out,text_in,text_out,abs_in,abs_out,rel_count,irel_count,in_count,out_count,url
0,"New Delhi: Calling for attacking corruption at the roots, Vice President M Venkaiah Naidu on Monday asked civil servants not to be misled by ""black sheep"" and render their service without fear or favour. Naidu also hailed GST as a revolutionary step and praised demonetisation. ""As you are new generation of civil servants, I hope you will not be misled by black sheep in the fold but would render your service without fear or favour and without any expectation of extraneous rewards. If you serve in the true spirit of service I am sure you will have your best reward,"" he said while inaugurat...",india_zee_states,"Monday, October 30, 2017, 20:22 +0530",Don't be misled by black sheep: Venkaiah Naidu tells civil servants,"{'civil': 1, 'servants': 1}",{},"{'civil': 1, 'servants': 1}",{},"{'attack': 1, 'corrupt': 5, 'president': 3, 'civil': 4, 'servants': 3, 'revolution': 1, 'vigilance': 5, 'laws': 1, 'reform': 3, 'minister': 1, 'strike': 1, 'govern': 2, 'welfare': 1}",{'education': 1},"{'attack': 1, 'corrupt': 1, 'president': 1, 'civil': 1, 'servants': 1}",{},9,0,13,1,http://zeenews.india.com/delhi/dont-be-misled-by-black-sheep-venkaiah-naidu-tells-civil-servants-2053007.html
1,"The UK's minister for agriculture has said he is hopeful that Brexit negotiations will produce a deal that is in everybody's interests. Michael Gove indicated that he plans to travel to Dublin next month for talks with members of the Government and with representatives of the Irish agricultural sector. He was speaking to RTÉ News following a meeting with the EU's Agriculture Commissioner, Phil Hogan, on the sidelines of a conference on the future of the dairy sector. Asked if the UK might be pursuing a treaty with the European Union that would give them a version of the current customs...",ireland_rte,2017-10-30T15:48:48UTC,Gove and Hogan discuss Brexit at dairy conference,{},{},{},{},"{'minister': 4, 'govern': 1, 'border': 1}","{'deal': 2, 'travel': 1, 'trade': 1}",{'minister': 1},"{'deal': 1, 'travel': 1}",1,2,3,3,http://www.rte.ie/news/2017/1030/916224-brexit/
2,"By EUOBSERVER Northern Ireland is heading towards an intervention from London after talks between Irish nationalist Sinn Fein and the pro-British DUP ended Friday without an agreement. Northern Ireland has been without a regional administration since January. London set Monday as a deadline for an agreement, to avoid a budget being introduced from Westminster. Northern Ireland state secretary James Brokenshire warned of ""a glide path to greater and greater UK government intervention"".",euroobs,"Mon, 30 Oct 2017 08:58:55 +0100",[Ticker] Deadline looms for Northern Ireland negotiations,{},{},{},{},"{'nationalist': 1, 'administration': 1, 'govern': 1}",{},{'nationalist': 1},{},1,0,3,0,https://euobserver.com/tickers/139693?utm_medium=rss
3,"Rohingya Refugee Women Bring Stories of Unspeakable Violence COX'S BAZAR, Bangladesh, Oct 10 2017 (IPS) - Yasmin, 26, holds her 10-day-old baby, who she gave birth to in a crowded refugee camp in Cox’s Bazar, a southeastern district bordering Myanmar. Three weeks ago, when she was still in her home in Hpaung Taw Pyin village in Myanmar, she was raped by a group of soldiers as houses burned, people fled and gunfire shattered the air. “I have been working as a human rights activist for the last 20 years but never heard of such an extreme level of violence."" --Bimol Chandra Dey Sarker, Chi...",ips_gender,2017-10-10T12:26:28UTC,Rohingya Refugee Women Bring Stories of Unspeakable Violence,"{'refuge': 1, 'violen': 1}",{},"{'refuge': 2, 'violen': 2}",{},"{'refuge': 12, 'violen': 7, 'border': 5, 'rape': 14, 'soldier': 8, 'human rights': 3, 'activist': 2, 'violat': 1, 'torture': 3, 'kill': 7, 'army': 2, 'guns': 1, 'govern': 1, 'military': 4, 'persecution': 1, 'genocid': 1, 'rulers': 1, 'assault': 2, 'unarmed': 2, 'abuse': 6, 'agenci

In [4]:
df1.describe()

,rel_count,irel_count,in_count,out_count
count,965802.000000,965802.000000,965802.000000,965802.000000
mean,3.435610,0.176660,10.322719,1.778942
std,2.660657,0.499922,7.219901,2.074872
min,0.000000,0.000000,1.000000,0.000000
25%,1.000000,0.000000,5.000000,0.000000
50%,3.000000,0.000000,8.000000,1.000000
75%,5.000000,0.000000,13.000000,2.000000
max,22.000000,11.000000,110.000000,62.000000


In [ ]:
初始的 效果不太好，微调
'''
remove_index =\
    ((relevant_count==0) & (irelevant_count > 0)) |\
    ((relevant_count >1) & (irelevant_count > 1)) |\
    ( text_in_count < 2) |\
    ((text_in_count >=2) & (text_in_count < text_out_count + 1))

remove_index = df1[remove_index== True].index
sizes.append(remove_index.shape[0])

df1 = df1[~df1.index.isin(remove_index)]
'''

In [36]:
# def get_matching_num(x):
#     x = [i.lower() for i in x]
#     return len(set(x))

In [5]:
relevant_count  = df1.rel_count 
irelevant_count = df1.irel_count 
text_in_count   = df1.in_count 
text_out_count  = df1.out_count 

# remove_index =\
#     ((relevant_count==0) & (irelevant_count > 0)) |\
#     (irelevant_count > 1)|\
#     ( text_in_count < 2) |\
#     ((text_in_count >=2) & (text_in_count < text_out_count * 2))

# remove_index = df1[remove_index== True].index
# print(remove_index.shape[0])

# df2 = df1[~df1.index.isin(remove_index)].reset_index(drop=True)


select_index = (relevant_count >= irelevant_count) & (text_in_count > text_out_count)
df2 = df1[select_index]

print(df1.shape[0], df2.shape[0])
# df2

965802 939613


In [10]:
df2.out_count.max()

62

In [11]:
df2[df2.out_count==62]

,text,source,date,title,title_in,title_out,url_in,url_out,text_in,text_out,abs_in,abs_out,rel_count,irel_count,in_count,out_count,url
708770,"In a year filled with heightened political vitriol, two deaths brought the nation together to remember men who represented a seemingly bygone era of U.S. politics. George H.W. Bush was a president, vice president, congressman, CIA director and Navy pilot during World War II, where he flew 58 missions and was shot down over the Pacific. As a politician, he had his fair share of critics and was voted out of office after one term as president. But the Republican reinvented himself in the years after his time in the White House, becoming a fundraiser for disaster relief and forming an unlikel...",minnstartrib_national,2018-12-17T18:55:38Z,Final goodbye: Roll call of some who died in 2018,{},{},{},{},"{'political': 10, 'death': 10, 'politics': 3, 'president': 21, 'congress': 3, 'war': 12, 'shot': 5, 'politician': 1, 'vote': 2, 'republican': 6, 'prison': 5, 'torture': 1, 'military': 4, 'propaganda': 1, 'battle': 2, 'kill': 3, 'convict': 1, 'civil': 3, 'rights': 3, 'senate': 3, 'force': 3, 'reform': 2, 'feminist': 1, 'army': 2, 'ambassador': 2, 'activist': 6, 'attack': 1, 'monarch': 2, 'minister': 5, 'party': 5, 'arrest': 1, 'govern': 11, 'supreme': 1, 'court': 3, 'dictator': 2, 'coup': 1, 'soldier': 1, 'guerrilla': 2, 'refuge': 2, 'bomb': 1, 'democratic': 2, 'liberal': 1, 'massacre': 1, ...","{'climate': 2, 'singer': 9, 'entertainment': 2, 'artist': 6, 'pop': 4, 'winning': 15, 'album': 1, 'career': 9, 'success': 6, 'film': 16, 'football': 1, 'sport': 4, 'music': 9, 'art': 1, 'weather': 1, 'fiction': 1, 'business': 5, 'grammy': 5, 'tv': 11, 'financ': 1, 'comed': 4, 'game': 1, 'econom': 3, 'broadway': 6, 'movie': 3, 'accident': 2, 'invest': 3, 'product': 1, 'baseball': 1, 'league': 1, 'oscar': 5, 'hollywood': 3, 'champion': 2, 'concert': 2, 'surfing': 1, 'player': 2, 'jazz': 3, 'culture': 4, 'olympic': 3, 'novels': 2, 'cinema': 1, 'fashion': 2, 'cardinals': 1, 'world series': 1, ...","{'political': 1, 'death': 1, 'politics': 1, 'president': 1}",{},4,0,74,62,http://www.startribune.com/nation/502953701.html


In [15]:
df2.columns

Index(['text', 'source', 'date', 'title', 'url', 'title_in', 'title_out',
       'url_in', 'url_out', 'text_in', 'text_out', 'abs_in', 'abs_out',
       'rel_count', 'irel_count', 'in_count', 'out_count'],
      dtype='object')

In [6]:
df2[['title','text','source','date','rel_count','in_count','irel_count','out_count','url']].to_csv('df2_419.csv', index=False)

In [7]:
df2[['title','text']].to_csv('df2_419_only_text.csv', index=False)

In [8]:
num_chunks = 100

df_save = np.array_split(df2[['title','text']], num_chunks)
save_chunks = lambda x: df_save[x].to_csv('step4b/%03d.csv'%x, index=False)
pandarallel.initialize(progress_bar=True)       
_ = pd.Series(range(num_chunks)).parallel_apply(save_chunks) 

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Save to json

In [9]:
df = pd.DataFrame(columns=['path','filename'])
df.path = [str(x) for x in pathlib.Path('step4b/').glob('**/*csv')]
df.iloc[:,1:] = df.path.str.replace('step4b/','').str.split('/', expand=True).values

df['json_file'] = df['filename'].apply(lambda x: x.replace('.csv', '.json'))
df['tar_file'] = 'tar419/'+ df.json_file+'.tar.gz'
df.json_file = 'json419/'+ df.json_file
df

,path,filename,json_file,tar_file
0,step4b/052.csv,052.csv,json419/052.json,tar419/052.json.tar.gz
1,step4b/007.csv,007.csv,json419/007.json,tar419/007.json.tar.gz
2,step4b/044.csv,044.csv,json419/044.json,tar419/044.json.tar.gz
3,step4b/046.csv,046.csv,json419/046.json,tar419/046.json.tar.gz
4,step4b/094.csv,094.csv,json419/094.json,tar419/094.json.tar.gz
...,...,...,...,...
95,step4b/084.csv,084.csv,json419/084.json,tar419/084.json.tar.gz
96,step4b/011.csv,011.csv,json419/011.json,tar419/011.json.tar.gz
97,step4b/081.csv,081.csv,json419/081.json,tar419/081.json.tar.gz
98,step4b/020.csv,020.csv,json419/020.json,tar419/020.json.tar.gz


In [10]:
df1 = pd.read_csv(df.path[0], header=0)
df1

,title,text
0,Mississippi man dies after shootout with police officer,"JACKSON, Miss. — A Mississippi man has died after a shootout with police. Jackson police spokesman Sgt. Roderick Holmes says 30-year-old Elliot Reed died Wednesday after an encounter with police at a south Jackson gas station. Holmes says Elliot and Chauncy Reed and the unnamed officer traded shots around 1 a.m. Elliot Reed was critically wounded and died hours later at a hospital. Chauncy Reed was arrested and is charged with aggravated assault on a police officer and capital murder. It's unclear to what crime the capital murder charges refer. The officer was not injured. Holmes says ..."
1,The Latest: Migrants head to Italy after standoff,"THESSALONIKI, Greece — The Latest on migrant issues in Europe (all times local): 8:35 p.m. An aid group is bringing a boat with 105 migrants to Sicily after a bureaucratic standoff by the Italian coast guard left the migrants exposed to the elements for over a day. The group SOS Mediterranee said Wednesday the passengers include 34 minors, including six children under 13, from nine countries. The migrants were rescued Sunday off Libya by the Spanish group Proactiva Open Arms, which hosted them on deck aboard their 30-foot (nine-meter) sailboat Astral. The group asked to transfer them ..."
2,"JD(U), RJD get ready for another by-election battle","more-in Swords were drawn for the Jokihat Assembly segment here on Wednesday with candidates of both the ruling Janata Dal (United) and the opposition RJD filing their nomination papers for May 28 bypoll. By-election on the seat has been necessitated due to resignation of sitting JD(U) MLA Sarfaraz Alam, who quit the party earlier this year and got elected - on an RJD ticket - to the Lok Sabha from Araria, a seat held by his late father Mohammad Taslimuddin. Tough fight While Alam’s younger brother Shahnawaz filed nomination papers as the RJD’s candidate, the JD(U) has fielded Murshid ..."
3,Soldiers Deployed To Fight Insurgency In Lake Chad Basin Allegedly Eloping With Young Women,"A new report by the United Nations Development Programme (UNDP), has revealed that some Nigerian soldiers and other troops sent to fight Boko Haram in the Lake Chad basin, are abandoning their duties to chase women. The UNDP report titled “Perspectives from local communities on stabilization and building peace in the Lake Chad Basin” which was released yesterday May 8th, said a lack of professionalism and reckless sexual behaviour, was exhibited by many of the security forces stationed in the communities around the lake. “In Cameroon, relations between security agencies and communities s..."
4,Middleman in Jayde Panayiotou murder to apply for bail,"PORT ELIZABETH - The self-confessed middleman, who helped arrange the murder of Port Elizabeth school teacher Jayde Panayiotou on behalf of her husband Christopher, appeared in the Port Elizabeth Magistrate’s Court on Wednesday. The case was postponed until May 18 for a formal bail application. Luthando Siyoni was arrested earlier this week after more than six months on the run. He is facing a charge of murder after his immunity from prosecution was revoked. Siyoni is being represented by Zolile Ngqeza who represented him as a Section 204 witness during Panayiotou’s murder trial last y..."
...,...,...
9391,Many on narcolist win barangay polls,"The so-called narcolist “helped a lot” in denying victory to candidates in Monday’s village polls who had been identified as involved in drugs, according to Metro Manila’s top cop. Election results, however, seemed to show otherwise. “I believe that the community was able to evaluate and assess, and they voted wisely,” said Director Camilo Cascolan, chief of the National Capital Region Police Office. ADVERTISEMENT The so-called narcolist from the Philippine Drug Enforcement Agency (PDEA) contains the names of 207 former and current village officials linked to the drug trade nationwide...."
9392,Boko Haram held Jonatha

In [22]:
# filename = df.path[0]
# json_file = df.json_file[0]
# tar_file = df.tar_file[0]

# df1 = pd.read_csv(filename, header=[0])
# df1.to_json(json_file)

# with tarfile.open(tar_file, "w:gz") as tar:
#     tar.add(json_file, arcname=os.path.basename(json_file))

In [11]:
def convert_json_tar(filename, json_file, tar_file):
    df1 = pd.read_csv(filename, header=[0])
    df1.to_json(json_file, orient="records", lines=True)
    
    with tarfile.open(tar_file, "w:gz") as tar:
        tar.add(json_file, arcname=os.path.basename(json_file))

pandarallel.initialize(nb_workers=16, progress_bar=True) 
_ = df.parallel_apply(lambda x: convert_json_tar(x['path'], x['json_file'], x['tar_file']), axis=1)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [12]:
pandarallel.initialize()
df['size'] = df['path'].parallel_apply(get_csv_size)
print(df['size'].sum())
#2209

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
2424.8
